In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from quspin.basis import spin_basis_1d
from quspin.operators import hamiltonian
from quspin.tools.evolution import ED_state_vs_time, expm_multiply_parallel
from quspin.tools.Floquet import Floquet_t_vec, Floquet
from tqdm import tqdm

plt.rcParams.update({
    "text.usetex": True,
})
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.fallback'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'
plt.style.use('seaborn-v0_8-deep')
prop_cycle = plt.rcParams['axes.prop_cycle']
dcolors = prop_cycle.by_key()['color']

# Model

The O'brian-Fendley model is much like the normal Majorana-Hubbard chain but with a slightly modified interaction term:

\begin{align*}
H_{OF} =& -it\sum_j \gamma_j \gamma_{j+1} -g \sum_j \gamma_{j-2}\gamma_{j-1}\gamma_{j+1}\gamma_{j+2}\\
=& t \sum_j \left(\sigma_j^x +\sigma_j^z \sigma_{j+1}^z\right)
+g \sum_j\left(\sigma_j^x \sigma_{j+1}^x \sigma_{j+2}^z + \sigma_j^z \sigma_{j+1}^z \sigma_{j+2}^z\right)
\end{align*}

In [ ]:
def drive(t, omega):
    return np.cos(omega*t)

def Hof_dynamic(omega, g, basis):
    """
    Hopping strength is cos(omega t) (time t is fed later to the Hamiltonian function)
    g is the coupling strength
    basis is quspin spinless_fermion_basis_1d
    """
    
    L = basis.N
    
    x_lst = [[-2, i] for i in range(L)]
    
    zz_lst = [[-2, i, i+1] for i in range(L-1)]
    
    xzz_lst = [[g, i, i+1, i+2] for i in range(L-2)]
    zzx_lst = [[g, i, i+1, i+2] for i in range(L-2)]

    zz_lst += [[-2, L-1, 0]]
    
    xzz_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    zzx_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    static = [['xzz', xzz_lst],
              ['zzx', zzx_lst]]
    dynamic = [['x', x_lst, drive, [omega]],
               ['zz', zz_lst, drive, [omega]]]
    
    H = hamiltonian(static, dynamic, basis=basis, check_symm=False, check_herm=False)
    return H

# Time evolution (with dynamical Hamiltonian)

I'm going to start with the $t=0$ ground state and evolve for a few periods using a built-in function from QuSpin that handles dynamic Hamiltonians by solving the Schrodinger equation. I don't know how efficient it is, so beware for large systems!

In [ ]:
L = 8
basis = spin_basis_1d(L, pauli=1) # pauli matrix, not spin 1/2 (matches iTensor "S=1/2")
g = 1
#omega = 1/np.sqrt(1+g**2)
omega = 100
H = Hof_dynamic(omega, g, basis)
e, v = H.eigsh(time=0, k=10, which='SA') # diagonalizing at time t=0
print(e[:10]) # +L*(t**2+g**2)/g)
v0 = v[:,0]
T = 2*np.pi/omega

In [ ]:
periods = 40
steps = 20*periods + 1 # 20 steps per period
times = np.linspace(0, periods*T, steps)
energies = np.zeros(steps)
olaps = np.zeros(steps, dtype=np.complex128)
vs = H.evolve(v0, 0, times, iterate=True)
for i, vi in tqdm(enumerate(vs), total=steps): # tqdm just makes a progress bar. Feel free to replace this line with enumerate(vs):
    t = times[i]
    #print(f"t/2 pi = {np.round(t/(2*np.pi), 3)}")
    energies[i] = H.expt_value(vi, time=t).real
    olaps[i] = np.vdot(vi, v0)
    #print(f"e = {energies[i]}, |overlap| = {np.abs(olaps[i])}")

In [ ]:
plt.figure(figsize=(6,4), dpi=200)
plt.plot(times/T, np.abs(olaps), '.', markerfacecolor='none')
plt.ylim(0, 1)
plt.xlim(0, times[-1]/T)
plt.xlabel(r'$t/T$')
plt.ylabel(r'$|\langle \psi(0)|\psi(t)\rangle|^2$')
plt.title(f'$L={L}$, $g/t={g}$, $\omega={omega}$, {periods} periods')
#plt.savefig(f'L{L}_g{g}_omega{omega}.pdf', bbox_inches='tight')